In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [15]:
data_files = {
	'inbound': '../Data/Inbound.csv',
	'outbound': '../Data/Outbound.csv',
	'inventory': '../Data/Inventory.csv',
	'operation': '../Data/OperationCost.csv',
	'material': '../Data/MaterialMaster.csv'
}

loaded_data = {}
for name, path in data_files.items():
	if not os.path.exists(path):
		print(f"Warning: File not found: {path}. Skipping loading '{name}'.")
		loaded_data[name] = None
	else:
		loaded_data[name] = pd.read_csv(path)

inbound = loaded_data['inbound']
outbound = loaded_data['outbound']
inventory = loaded_data['inventory']
operation = loaded_data['operation']
material = loaded_data['material']

<h2>Used PowerBI to create visualizations for making insights.</h2>

Top 5 materials based on inbound amount (MT): MAT-0145, MAT-0332, MAT-0035, MAT-0144, MAT-0112

In [ ]:
inventory['BALANCE_AS_OF_DATE'] = pd.to_datetime(inventory['BALANCE_AS_OF_DATE'])
today = pd.to_datetime('2024-12-31')

# Merge with MaterialMaster
inventory = inventory.drop(columns=['SHELF_LIFE_IN_MONTH'], errors='ignore')
inventory = inventory.merge(material[['MATERIAL_NAME', 'SHELF_LIFE_IN_MONTH', 'DOWNGRADE_VALUE_LOST_PERCENT']], 
                           on='MATERIAL_NAME', how='left')

# Inventory Age and Risk Level
inventory['Inventory_Age_Months'] = (today - inventory['BALANCE_AS_OF_DATE']).dt.days // 30
inventory['Risk_Level'] = np.where(inventory['SHELF_LIFE_IN_MONTH'].isna(), 'Unknown',
                                  np.where(inventory['Inventory_Age_Months'] >= inventory['SHELF_LIFE_IN_MONTH'], 'Expired',
                                           np.where(inventory['Inventory_Age_Months'] >= inventory['SHELF_LIFE_IN_MONTH'] * 0.8, 'High Risk', 'Safe')))

# Financial Loss
inventory['Financial_Loss'] = np.where(inventory['Risk_Level'] == 'Expired',
                                      inventory['STOCK_SELL_VALUE'].fillna(0) * (inventory['DOWNGRADE_VALUE_LOST_PERCENT'].fillna(50) / 100),
                                      0)

# Summarize risks by plant
risk_summary = inventory.groupby(['PLANT_NAME', 'Risk_Level']).agg({'UNRESRICTED_STOCK': 'sum', 'Financial_Loss': 'sum'}).reset_index()

# Insight
print("Risk Summary by Plant:")
print(risk_summary)
max_loss_plant = risk_summary.loc[risk_summary['Financial_Loss'].idxmax()]['PLANT_NAME']
print(f"\nPriority Action: Address {max_loss_plant} with highest financial loss.")

Risk Summary by Plant:
            PLANT_NAME Risk_Level  UNRESRICTED_STOCK  Financial_Loss
0      CHINA-WAREHOUSE    Expired          396379268     704186998.4
1      CHINA-WAREHOUSE  High Risk           46631299             0.0
2      CHINA-WAREHOUSE       Safe          307943232             0.0
3  SINGAPORE-WAREHOUSE    Expired          305674385      81665905.1
4  SINGAPORE-WAREHOUSE  High Risk           26399585             0.0
5  SINGAPORE-WAREHOUSE       Safe          228419784             0.0

Priority Action: Address CHINA-WAREHOUSE with highest financial loss.
